In [8]:
import os
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.display import display, HTML

display(HTML("<style>.container { width: 90% !important}; </style>"))

In [10]:
np.random.seed(1)
%matplotlib inline